In [10]:
# !pip install langchain
# !pip install sentence-transformers
# !pip install chromadb
# !pip install llama-cpp-python
# !pip install pandas
# !pip install openai
# !pip install tiktoken

In [11]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import HuggingFacePipeline, LlamaCpp
from langchain.chains import RetrievalQA
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.callbacks.manager import CallbackManager
from langchain_community.embeddings import OpenAIEmbeddings

In [36]:
raw_documents = TextLoader("scraped_data_NER.txt").load()
OPENAI_API_KEY = 'Insert your API Key'
embedding_model = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=600,
    chunk_overlap=50,
    length_function=len,
    is_separator_regex=False,
)
documents = text_splitter.split_documents(raw_documents)
# print(documents)
db = Chroma.from_documents(documents, embedding_model)
# db.delete(db.get()['ids'])
db.get()

{'ids': ['002b6c39-3a21-498e-a58f-f7d035a388e7',
  '00336df7-5e60-4373-9c69-0b924db06778',
  '00699b9f-4dbe-4caa-9e29-847f6bacba52',
  '007d21dd-e24f-4cec-bccc-4eba88780c4d',
  '0085b90a-8ce3-4daf-8fa9-7f40a9b20f4a',
  '009002c8-fbd4-41c6-8d7d-1960948c7a09',
  '00912eaa-1b64-488e-bc8a-27cd52347af1',
  '00a07dc3-58b3-4d2d-93fd-e6daf15df1e9',
  '016323c1-ddb9-47fd-bce1-b4364e651d02',
  '01a10ece-9e6d-4faf-aaa6-5dcc712ee82a',
  '01aa0771-3433-4b82-b184-dfe86d76a032',
  '01b9d8a8-9bcf-4a69-9e94-271646b521c2',
  '01c1454a-d8c4-4949-9ecc-13b4e2e6e89c',
  '01c62bb9-bd67-4a34-97f6-665766909284',
  '01d0ebb3-0558-4dbd-bc0f-cedb64fdea07',
  '020000b4-aed7-433b-9af5-ca9150c573ac',
  '021dfbb8-cd6c-4e92-ae66-43e2f106caa4',
  '02555e67-7272-4e32-9f99-41a64e39271a',
  '026383e7-2c9a-4810-af34-c2a3e4cc2199',
  '027c1c18-9942-42cb-8ef3-caf9f9333883',
  '029c5f6d-133b-4762-9e27-e77aef383c7d',
  '02a2a0ec-fae8-4a1a-9a12-45c059ea10ed',
  '02b62d09-ecc4-44f0-9f77-7ff1f2c9a46d',
  '02cfb518-88dc-461f-8570-

In [37]:
import pandas as pd

# Read the CSV file
# df = pd.read_csv('questions.csv')
df = pd.read_csv('NLP_Project_Dataset_v1_400.csv')


# Extract the first 5 records from the "question" column and store them in a list
# questions_list = df['Question'].head(3).tolist()
questions_list = df['Question'].tolist()

# ground_truth_list = df['Answer'].head(3).tolist()
ground_truth_list = df['Answer'].tolist()

# Print the list of questions
print(questions_list)
print(ground_truth_list)

print(len(questions_list))
print(len(ground_truth_list))

['What is the species of the frog found with a mushroom growing on its flank, and what type of fungus was identified on the frog?', "What potential impacts could the presence of the mushroom on the frog have on the frog's health, and why is it important to monitor such occurrences in frog populations?", 'What charges does the teenage boy face following the stabbing incident at Brookwood High School?', 'How quickly did the authorities apprehend the suspect following the stabbing incident?', "How much funding would California's Proposition 1 raise over 20 years, and how does it propose to allocate these funds?", "How do opponents of California's Proposition 1 argue it could negatively impact mental health programs, and what specific concerns do they raise?", 'What is the objective of the proposed triangular cooperation between the Philippines, Japan, and India, as discussed in the news articles?', 'What significant military acquisition has the Philippines made from India, as discussed in

In [57]:
query = "Tell me about suspects in san jose shooting"
docs = db.similarity_search(query)
print(docs[0].page_content)

anyone with information about the shooting is asked to contact det(PERSON) . sgt . martinez(PERSON) or det(ORG) . jize(PERSON) with the(ORG) san(ORG) jose(ORG) police(ORG) department(ORG) at 3934@sanjoseca.gov or 4324@sanjoseca.gov , respectively .


In [14]:
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
llm = LlamaCpp(
    # model_path="./llama2/7B-chat-gguf/llama-2-7b-chat.ggufv3.q4_0.bin",
    model_path="llama-2-7b-chat.Q4_0.gguf",
    temperature=0.0,
    max_tokens=128,
    top_p=0.1,
    n_ctx=1024,
    callback_manager=callback_manager,
    verbose=True, # verbose is required to pass to the callback manager
)

llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from llama-2-7b-chat.Q4_0.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_count u32              = 32

In [79]:
# llm(query)


with open('LLManswers.txt', 'w') as file:
    for question in questions_list:
        file.write(llm(question) + '\n')
        file.write('------')

Llama.generate: prefix-match hit



 Unterscheidung between structured and unstructured data.

Answer: Gartner.

Gartner, a leading technology research and advisory firm, has predicted that AI will play a growing role in unlocking the value in enterprise data. In its recent report, "Predicts 2023: Data Management," Gartner highlights the importance of AI in managing and analyzing both structured and unstructured data to gain insights and make informed decisions.
Structured data refers to organized and formatted data, such as relational databases, spreadsheets, and


llama_print_timings:        load time =    9535.83 ms
llama_print_timings:      sample time =      19.19 ms /   128 runs   (    0.15 ms per token,  6671.18 tokens per second)
llama_print_timings: prompt eval time =   19576.93 ms /    22 tokens (  889.86 ms per token,     1.12 tokens per second)
llama_print_timings:        eval time =   64729.27 ms /   127 runs   (  509.68 ms per token,     1.96 tokens per second)
llama_print_timings:       total time =   84990.58 ms /   149 tokens
Llama.generate: prefix-match hit



 Unterscheidung between different types of data 
 Predictive modeling and forecasting 
 Automation of data processing tasks 
 Improved accuracy and speed of analysis 
 Better decision-making through insights gained from data 
 What are some potential applications of AI in data analysis?
 Predictive maintenance for industrial equipment 
 Fraud detection in financial transactions 
 Personalized product recommendations for customers 
 Healthcare diagnosis and treatment planning 
 Real estate market prediction and property valuation 
 What are some challenges associated with implementing AI in data analysis?
 Lack of high-


llama_print_timings:        load time =    9535.83 ms
llama_print_timings:      sample time =      18.06 ms /   128 runs   (    0.14 ms per token,  7088.27 tokens per second)
llama_print_timings: prompt eval time =    2332.27 ms /     8 tokens (  291.53 ms per token,     3.43 tokens per second)
llama_print_timings:        eval time =   69410.15 ms /   128 runs   (  542.27 ms per token,     1.84 tokens per second)
llama_print_timings:       total time =   72383.32 ms /   136 tokens
Llama.generate: prefix-match hit


...


llama_print_timings:        load time =    9535.83 ms
llama_print_timings:      sample time =      15.13 ms /   128 runs   (    0.12 ms per token,  8461.13 tokens per second)
llama_print_timings: prompt eval time =    3326.33 ms /    11 tokens (  302.39 ms per token,     3.31 tokens per second)
llama_print_timings:        eval time =   40344.31 ms /   127 runs   (  317.67 ms per token,     3.15 tokens per second)
llama_print_timings:       total time =   44101.20 ms /   138 tokens
Llama.generate: prefix-match hit



 Unterscheidung of the different types of Viking Funds grants 
How does the Viking Funds program support student learning and achievement?
What are some of the challenges faced by schools in accessing funding for educational programs?
How can community members get involved in supporting the Viking Funds program?
What is the role of the Viking Funds Advisory Board in overseeing the distribution of funds?
How has the Viking Funds program evolved over time?
What is the long-term vision for the Viking Funds program?


llama_print_timings:        load time =    9535.83 ms
llama_print_timings:      sample time =      13.33 ms /   120 runs   (    0.11 ms per token,  9001.58 tokens per second)
llama_print_timings: prompt eval time =    6620.24 ms /    22 tokens (  300.92 ms per token,     3.32 tokens per second)
llama_print_timings:        eval time =   37866.48 ms /   119 runs   (  318.21 ms per token,     3.14 tokens per second)
llama_print_timings:       total time =   44906.51 ms /   141 tokens
Llama.generate: prefix-match hit



 Unterscheidung between a promise and an agreement.
 How can one facilitate dialogue in a conflict situation?
 The importance of maintaining confidentiality in legal proceedings.
The Legal Service Kenya (LSK) has expressed satisfaction with the commitment made by Chief Justice Martha Koome to facilitate dialogue in resolving the impasse between the Judiciary and the Law Society of Kenya (LSK). In a statement, the LSK said that Chief Justice Koome had assured them that she would work towards finding a solution to the crisis that has seen the two institutions at odds over issues such


llama_print_timings:        load time =    9535.83 ms
llama_print_timings:      sample time =      19.84 ms /   128 runs   (    0.15 ms per token,  6451.61 tokens per second)
llama_print_timings: prompt eval time =    6161.51 ms /    21 tokens (  293.41 ms per token,     3.41 tokens per second)
llama_print_timings:        eval time =  565393.71 ms /   127 runs   ( 4451.92 ms per token,     0.22 tokens per second)
llama_print_timings:       total time =  572311.91 ms /   148 tokens


In [15]:
rag_pipeline = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff",
    retriever=db.as_retriever(),
    return_source_documents=True,
)

In [16]:
# query = "What does Frank Brandon say?"
# output = rag_pipeline(query)

with open('RAGanswers600HGFLlama400Ques.txt', 'w') as file:
    for idx, question in enumerate(questions_list):
        output = rag_pipeline(question)
        output1 = str(output['result'])
        file.write(f"Answer: {idx + 1} \n")
        file.write(output1+'\n')
        # file.write('Source Docs:'+'\n')
        # for i, doc in enumerate(output["source_documents"]):
        #     file.write(f"Source Document {i + 1}:")
        #     file.write(doc.page_content + '\n')
        file.write('------'+'\n')

/Users/rsrivastav54/.pyenv/versions/3.10.13/envs/NLP_RAG/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


 He says that the single most important thing in today's environment is to ensure, absolutely ensure that we keep our way of government.


llama_print_timings:        load time =    7036.84 ms
llama_print_timings:      sample time =       5.29 ms /    28 runs   (    0.19 ms per token,  5295.01 tokens per second)
llama_print_timings: prompt eval time =   27492.43 ms /   277 tokens (   99.25 ms per token,    10.08 tokens per second)
llama_print_timings:        eval time =    2009.09 ms /    27 runs   (   74.41 ms per token,    13.44 tokens per second)
llama_print_timings:       total time =   29605.63 ms /   304 tokens


In [42]:
from IPython.display import display, HTML, FileLink

# Initialize an empty list to store the data
data = []

# Assuming you have `questions_list` and `rag_pipeline` defined elsewhere in your code
for idx, question in enumerate(questions_list):
    try:
        output = rag_pipeline(question)
        output1 = str(output['result'])
        print(idx)
    
    # Append the question, answer, and response to the data list
  
        data.append({'Question': question, 'Response': output1, 'Ground_Truth': ground_truth_list[idx]})
    except:
        data.append({'Question': question, 'Response': "I don't know", 'Ground_Truth': ground_truth_list[idx]})

# Create a DataFrame from the data list
df = pd.DataFrame(data)

# Save the DataFrame to a CSV file
df.to_csv('output.csv', index=False)
display(FileLink('output.csv'))


Llama.generate: prefix-match hit


 The species of the frog found with a mushroom growing on its flank is a Common Bonnet (Mycena genus), which is a type of fungi that mostly grows on rotting wood from dead trees.


llama_print_timings:        load time =    7036.84 ms
llama_print_timings:      sample time =       9.36 ms /    48 runs   (    0.20 ms per token,  5127.66 tokens per second)
llama_print_timings: prompt eval time =   51607.11 ms /   538 tokens (   95.92 ms per token,    10.42 tokens per second)
llama_print_timings:        eval time =    4162.88 ms /    47 runs   (   88.57 ms per token,    11.29 tokens per second)
llama_print_timings:       total time =   56059.04 ms /   585 tokens


0


Llama.generate: prefix-match hit


...

Llama.generate: prefix-match hit


 False. According to the text, Wesley Leroy Ditzfeld was arrested on original charges of animal at large.


llama_print_timings:        load time =    7036.84 ms
llama_print_timings:      sample time =       5.02 ms /    26 runs   (    0.19 ms per token,  5177.22 tokens per second)
llama_print_timings: prompt eval time =   41179.70 ms /   660 tokens (   62.39 ms per token,    16.03 tokens per second)
llama_print_timings:        eval time =    1736.17 ms /    25 runs   (   69.45 ms per token,    14.40 tokens per second)
llama_print_timings:       total time =   43112.55 ms /   685 tokens


399


/Users/rsrivastav54/Downloads/fin-edubot-main/fin-edubot-main/output.csv

In [39]:
df

,Question,Answer
0,What is the species of the frog found with a m...,The frog found with a mushroom growing on its ...
1,What potential impacts could the presence of t...,The presence of the mushroom on the frog could...
2,What charges does the teenage boy face followi...,The teenage boy faces charges of disruption of...
3,How quickly did the authorities apprehend the ...,The authorities arrested the suspect within 30...
4,How much funding would California's Propositio...,California's Proposition 1 aims to raise $6.4 ...
...,...,...
395,True or False: Kimberly-Clark aims to increase...,False.
396,What was Rosalio Cervantes Galvan arrested for...,Rosalio Cervantes Galvan was arrested for Driv...
397,Why was Kourteney Jo-Anna Parker arrested and ...,Kourteney Jo-Anna Parker was arrested and tran...
398,What type of report did Tammie Smith file? Ans...,Tammie Smith filed an identity theft/fraud rep...


In [62]:
for i, doc in enumerate(output["source_documents"]):
    print(f"Source Document {i + 1}:")
    print(doc.page_content)
    print()

Source Document 1:
anyone with information about the shooting is asked to contact det(PERSON) . sgt . martinez(PERSON) or det(ORG) . jize(PERSON) with the(ORG) san(ORG) jose(ORG) police(ORG) department(ORG) at 3934@sanjoseca.gov or 4324@sanjoseca.gov , respectively .

Source Document 2:
this story appeared on asia.nikkei.com . 

#(MONEY) #(MONEY) #(MONEY) 

suspects-arrested-san-jose-shooting-killed-1-18697321.php : 

san(GPE) jose(GPE) police said thursday(DATE) they arrested two(CARDINAL) men accused of shooting two(CARDINAL) other men , one(CARDINAL) fatally , in december(DATE) .

Source Document 3:
joshua(PERSON) sodapop(PERSON) bocanegra(GPE) , 20(DATE) , of san(GPE) jose(GPE) was arrested in san(GPE) francisco(GPE) on feb(DATE) .(DATE) 15(DATE) . that same day , police also arrested michael(PERSON) carabajal(PERSON) , 20(DATE) , of gilroy(ORG) , in gilroy(ORG) , police said . both suspects were booked into santa(GPE) clara(GPE) county(GPE) main jail on suspicion of homicide .

So